In [1]:
from obspy import Trace, Stream, UTCDateTime
from zizou.util import generate_test_data, test_signal
from datetime import timedelta, datetime
import os
import numpy as np
import requests

In [2]:

doi = "https://doi.org/10.5281/zenodo.13377159"
r = requests.get(doi)


In [4]:
r.ok

True

In [8]:
r.url
recordID = r.url.split("/")[-1]
recordID

'13377159'

In [9]:
zenodo_url = "https://zenodo.org/api/records/"
r = requests.get(zenodo_url + recordID)

In [15]:
import requests
import tarfile

def download_and_extract_tar(url, extract_path):
    # Download the tar file
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open('file.tar', 'wb') as f:
            f.write(response.raw.read())

        # Open the tar file
        tar = tarfile.open('file.tar')

        # Extract it
        tar.extractall(path=extract_path)
        tar.close()
    else:
        print(f"Failed to download {url}")

# Use the function
url = "https://zenodo.org/records/13377159/files/zizou_test_data.tar.gz"
download_and_extract_tar(url, '/tmp')

In [14]:
js = r.json()
js['files'][0]['key']

'zizou_test_data.tar.gz'

In [4]:

tr = test_signal(nsec=4320*600., sampling_rate=1/600.,
                 starttime=UTCDateTime(2020, 7, 18, 8, 22))
tr

NZ.BLUB..HHZ | 2020-07-18T08:22:00.000000Z - 2020-08-17T08:12:00.000000Z | 600.0 s, 4320 samples

In [5]:
start = tr.stats.starttime
end = tr.stats.endtime
sds_fmtstr = os.path.join(
    "{year}", "{network}", "{station}", "{channel}.{sds_type}",
    "{network}.{station}.{location}.{channel}.{sds_type}.{year}.{doy:03d}")
rootdir = "/tmp/sds_test"


current_date = start.date
while current_date <= end.date:
    print(UTCDateTime(current_date))
    _tr = tr.slice(UTCDateTime(current_date), UTCDateTime(current_date) + 86400)
    print(_tr.stats.starttime, _tr.stats.endtime)
    fullpath = sds_fmtstr.format(year=_tr.stats.starttime.year,
                                 doy=_tr.stats.starttime.julday,
                                 sds_type="D", **_tr.stats)
    fullpath = os.path.join(rootdir, fullpath)
    dirname, filename = os.path.split(fullpath)
    os.makedirs(dirname, exist_ok=True)
    print("writing", fullpath)
    _tr.write(fullpath, format="MSEED")
    current_date += timedelta(days=1)
 

2020-07-18T00:00:00.000000Z
2020-07-18T08:22:00.000000Z 2020-07-19T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.200
2020-07-19T00:00:00.000000Z
2020-07-19T00:02:00.000000Z 2020-07-20T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.201
2020-07-20T00:00:00.000000Z
2020-07-20T00:02:00.000000Z 2020-07-21T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.202
2020-07-21T00:00:00.000000Z
2020-07-21T00:02:00.000000Z 2020-07-22T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.203
2020-07-22T00:00:00.000000Z
2020-07-22T00:02:00.000000Z 2020-07-23T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.204
2020-07-23T00:00:00.000000Z
2020-07-23T00:02:00.000000Z 2020-07-24T00:02:00.000000Z
writing /tmp/sds_test/2020/NZ/BLUB/HHZ.D/NZ.BLUB..HHZ.D.2020.205
2020-07-24T00:00:00.000000Z
2020-07-24T00:02:00.000000Z 2020-07-25T00:02:00.000000Z
writing /tmp/sds_test/

In [6]:
from obspy.clients.filesystem.sds import Client

In [7]:
c = Client(rootdir)

In [8]:
st = c.get_waveforms("NZ", "BLUB", "", "HHZ", UTCDateTime(2020, 7, 1, 8, 22), UTCDateTime(2020, 8, 23, 8, 23))

In [9]:
tr_test = st.merge(fill_value=np.nan)[0]

In [10]:
np.alltrue(tr.data == tr_test.data)

True

In [13]:
st = c.get_waveforms("NZ", "MMM", "", "HHZ", UTCDateTime(2024, 7, 1, 8, 22), UTCDateTime(2024, 8, 23, 8, 23))

In [26]:
tr_test.stats['cached'] = True
tr_test.stats.cached

True

In [22]:
(datetime.now() - datetime(2024, 8, 25))/timedelta(seconds=1)

94807.655344